# ML-driven SL/TP/BE/Trailing Explorer
複数パラメータセットのBT結果を集計し、次の候補を提案するためのノートです。

## 概要
- `analysis/bt_results` の TradeLog を読み込み、SL/TP/BE/Trailing を含むパラメータセットごとに集計します。
- ランダム/グリッド/Bayesian (scikit-optimize が入っている場合) で次の候補を提案します。
- 推奨パラメータを CSV に書き出し、MT4 テスター用に使います。

In [ ]:

import json
import itertools
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Any
import pandas as pd
import numpy as np

BT_DIR = Path('analysis/bt_results')
PARAM_EXPORT_DIR = Path('analysis/param_candidates')
PARAM_EXPORT_DIR.mkdir(parents=True, exist_ok=True)
RNG = np.random.default_rng(42)
print('BT_DIR:', BT_DIR)
print('PARAM_EXPORT_DIR:', PARAM_EXPORT_DIR)


In [ ]:

# ==== BTログの読み込みユーティリティ ====

def list_bt_logs(pattern='TradeLog_*.csv'):
    return sorted(BT_DIR.glob(pattern))


def load_bt_log(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'].str.replace('.', '-', regex=False))
    df['source_log'] = path.name
    return df


def load_all_bt_logs(pattern='TradeLog_*.csv'):
    files = list_bt_logs(pattern)
    if not files:
        raise FileNotFoundError('BTログが見つかりません')
    frames = [load_bt_log(p) for p in files]
    return pd.concat(frames, ignore_index=True)

bt_df = load_all_bt_logs()
print('loaded rows:', len(bt_df), 'from', bt_df['source_log'].nunique(), 'logs')


In [ ]:

# ==== 評価指標関数 ====

def compute_metrics(df: pd.DataFrame) -> dict:
    exits = df[df['event'].str.upper() == 'EXIT'].copy()
    exits['net'] = pd.to_numeric(exits['net'], errors='coerce')
    exits = exits.dropna(subset=['net'])
    if exits.empty:
        return {
            'trades': 0,
            'wins': 0,
            'losses': 0,
            'win_rate_pct': np.nan,
            'pf': np.nan,
            'expectancy': np.nan,
            'max_dd': np.nan,
        }
    exits = exits.sort_values('timestamp')
    total = len(exits)
    wins = exits['net'] > 0
    losses = exits['net'] < 0
    gross_profit = exits.loc[wins, 'net'].sum()
    gross_loss = exits.loc[losses, 'net'].sum()
    pf = gross_profit / abs(gross_loss) if gross_loss != 0 else np.inf
    win_rate = wins.sum() / total if total else np.nan
    expectancy = exits['net'].mean()
    equity = exits['net'].cumsum()
    equity = pd.concat([pd.Series([0.0]), equity], ignore_index=True)
    peak = equity.cummax()
    dd = equity - peak
    max_dd = dd.min()
    return {
        'trades': int(total),
        'wins': int(wins.sum()),
        'losses': int(losses.sum()),
        'win_rate_pct': win_rate * 100,
        'pf': pf,
        'expectancy': expectancy,
        'max_dd': max_dd,
    }


In [ ]:

# ==== パラメータ空間の定義 (例) ====
param_space = {
    'stop_loss_pips': [80, 100, 120],
    'take_profit_pips': [120, 160, 200],
    'breakeven_trigger_pips': [30, 40, 50],
    'breakeven_offset_pips': [0, 10, 20],
    'trail_trigger_pips': [40, 60, 80],
    'trail_step_pips': [10, 15, 20],
}
print('param_space keys:', list(param_space))


In [ ]:

# ==== 候補セット生成 (グリッド or ランダム) ====

def generate_grid(space: Dict[str, List[Any]], limit: int = None):
    keys = list(space)
    values = [space[k] for k in keys]
    combos = itertools.product(*values)
    rows = []
    for i, combo in enumerate(combos):
        if limit and i >= limit:
            break
        row = {k: v for k, v in zip(keys, combo)}
        row['candidate_id'] = f'grid_{i:04d}'
        rows.append(row)
    return pd.DataFrame(rows)


def generate_random(space: Dict[str, List[Any]], n_samples: int):
    keys = list(space)
    rows = []
    for i in range(n_samples):
        row = {k: RNG.choice(space[k]) for k in keys}
        row['candidate_id'] = f'rand_{i:04d}'
        rows.append(row)
    return pd.DataFrame(rows)

# 例: グリッドの先頭50件 + ランダム50件
candidates = pd.concat([
    generate_grid(param_space, limit=50),
    generate_random(param_space, n_samples=50)
], ignore_index=True)
print('candidates', candidates.shape)
candidates.head()


In [ ]:

# ==== BT結果とパラメータの紐付け ====
# 前提: TradeLog ファイル名や run_id に candidate_id を含めておく (例: TradeLog_v125_..._rand_0001.csv)
# 下のマッピングは必要に応じてカスタマイズしてください。

def extract_candidate_id(name: str):
    # 例: ファイル名に grid_0001 や rand_0001 が含まれていると仮定
    import re
    m = re.search(r'(grid_\d{4}|rand_\d{4})', name)
    return m.group(1) if m else None

bt_df['candidate_id'] = bt_df['source_log'].apply(extract_candidate_id)
missing = bt_df['candidate_id'].isna().mean() * 100
print(f'candidate_id 未検出率: {missing:.1f}%')


In [ ]:

# ==== 候補ごとの指標集計 ====
metrics_rows = []
for cid, group in bt_df.groupby('candidate_id'):
    if pd.isna(cid):
        continue
    metrics = compute_metrics(group)
    metrics['candidate_id'] = cid
    metrics_rows.append(metrics)
metrics_df = pd.DataFrame(metrics_rows)
print('metrics rows:', len(metrics_df))
metrics_df.head()


In [ ]:

# ==== 評価スコア (例) ====
# 期待値とPFを重視しつつ、最大DDが深いものは減点するシンプルなスコア例。

def score_row(row, pf_weight=1.0, exp_weight=1.0, dd_penalty=0.5):
    pf = row.get('pf', np.nan)
    exp = row.get('expectancy', np.nan)
    dd = row.get('max_dd', 0)
    if pd.isna(pf) or pd.isna(exp):
        return -np.inf
    return pf_weight * pf + exp_weight * exp + dd_penalty * dd  # ddはマイナス値なので加点にするとペナルティになる

metrics_df['score'] = metrics_df.apply(score_row, axis=1)
metrics_df.sort_values('score', ascending=False).head(10)


In [ ]:

# ==== 次の候補をBayesian Optimizationで提案 (scikit-optimizeがある場合のみ) ====
try:
    from skopt.space import Real, Integer
    from skopt import gp_minimize

    search_space = [
        Integer(min(param_space['stop_loss_pips']), max(param_space['stop_loss_pips']), name='stop_loss_pips'),
        Integer(min(param_space['take_profit_pips']), max(param_space['take_profit_pips']), name='take_profit_pips'),
        Integer(min(param_space['breakeven_trigger_pips']), max(param_space['breakeven_trigger_pips']), name='breakeven_trigger_pips'),
        Integer(min(param_space['breakeven_offset_pips']), max(param_space['breakeven_offset_pips']), name='breakeven_offset_pips'),
        Integer(min(param_space['trail_trigger_pips']), max(param_space['trail_trigger_pips']), name='trail_trigger_pips'),
        Integer(min(param_space['trail_step_pips']), max(param_space['trail_step_pips']), name='trail_step_pips'),
    ]

    def objective(x):
        # 既存metricsがあれば近いものを補間、無ければペナルティ
        cols = ['stop_loss_pips','take_profit_pips','breakeven_trigger_pips','breakeven_offset_pips','trail_trigger_pips','trail_step_pips']
        point = dict(zip(cols, x))
        # 最近傍のスコアを返す簡易ロジック（実際は再BTで置き換える）
        if not candidates.empty:
            tmp = candidates.copy()
            for c in cols:
                tmp[c+'_diff'] = (tmp[c] - point[c]).abs()
            tmp['dist'] = tmp[[c+'_diff' for c in cols]].sum(axis=1)
            best = tmp.loc[tmp['dist'].idxmin()]
            cid = best['candidate_id']
            match = metrics_df[metrics_df['candidate_id'] == cid]
            if not match.empty:
                return -float(match['score'].iloc[0])
        return 0.0  # 未評価はフラット

    res = gp_minimize(objective, search_space, n_calls=10, random_state=42)
    suggested = dict(zip([s.name for s in search_space], res.x))
    suggested['candidate_id'] = f'bo_{len(candidates):04d}'
    suggested_df = pd.DataFrame([suggested])
    print('Suggested (BO):', suggested_df)
except ImportError:
    print('scikit-optimize が無いため BO 提案はスキップしました')
    suggested_df = pd.DataFrame()


In [ ]:

# ==== 推奨パラメータの出力 ====

def export_candidates(df: pd.DataFrame, filename: str):
    path = PARAM_EXPORT_DIR / filename
    df.to_csv(path, index=False)
    print('wrote', path)

# 推奨: スコア上位5件 + BO提案があれば併合
recommend_top = metrics_df.sort_values('score', ascending=False).head(5).merge(candidates, on='candidate_id', how='left')
output_df = pd.concat([recommend_top, suggested_df], ignore_index=True, sort=False)
export_candidates(output_df, 'recommended_params.csv')


## 使い方
1. `param_space` を調整し、候補IDがファイル名に入るようBTを実行。
2. TradeLogを `analysis/bt_results` に配置。
3. このノートを実行して `recommended_params.csv` を生成。
4. 推奨パラメータで再BTし、ウォークフォワードで確認。

In [ ]:

# ==== AtrBandConfig 生成 (推奨パラメータを指標ごとに反映) ====
# 前提:
# - recommended_params.csv に候補が出力済み
# - ベースとなるAtrBandConfigを読み込み、指定指標のSL/TP/BE/Trail列を置換する
# - pips値をそのまま倍率として扱う簡易版。必要なら換算ロジックを差し替えてください。

import pandas as pd
from pathlib import Path

# 入力設定
BASE_CONFIG = Path('../Config/v125_AtrBandConfig_YoYoEA_Multi_Entry_Test_All_CCI_Fix_MA_Fix_RSI_Fix_Sto3.csv')
RECOMMENDED_PATH = Path('../analysis/param_candidates/recommended_params.csv')
OUTPUT_CONFIG = BASE_CONFIG.with_name(BASE_CONFIG.stem + '_from_ml.csv')

# どの候補行を使うか (デフォルト: 先頭1件)
RECOMMENDED_INDEX = 0
# どの指標に反映するか
TARGET_INDICATORS = ['MA', 'RSI', 'CCI', 'MACD', 'STOCH']

# pips値を倍率に変換する場合の係数 (今はそのまま入れる)
PIPS_TO_ATR_FACTOR = 1.0

# 読み込み
rec_df = pd.read_csv(RECOMMENDED_PATH)
if RECOMMENDED_INDEX >= len(rec_df):
    raise IndexError('RECOMMENDED_INDEX が recommended_params の範囲外です')
rec = rec_df.iloc[RECOMMENDED_INDEX]

base_cfg = pd.read_csv(BASE_CONFIG)
out_cfg = base_cfg.copy()

# 推奨値を抽出（存在する場合のみ）
val = lambda k: rec[k] if k in rec and pd.notna(rec[k]) else None
sl = val('stop_loss_pips')
tp = val('take_profit_pips')
be_trig = val('breakeven_trigger_pips')
be_off = val('breakeven_offset_pips')
trail_trig = val('trail_trigger_pips')
trail_step = val('trail_step_pips')

for ind in TARGET_INDICATORS:
    # Enable/Modeはそのまま。SL/TP/BE/Trailだけ置換。
    if sl is not None:
        out_cfg[f'{ind}_SL'] = float(sl) * PIPS_TO_ATR_FACTOR
    if tp is not None:
        out_cfg[f'{ind}_TP'] = float(tp) * PIPS_TO_ATR_FACTOR
    if be_trig is not None:
        out_cfg[f'{ind}_BE_Enable'] = 'ON'
        out_cfg[f'{ind}_BE_Atr'] = float(be_trig) * PIPS_TO_ATR_FACTOR
    if be_off is not None:
        out_cfg[f'{ind}_BE_Offset'] = float(be_off)
    if trail_trig is not None:
        out_cfg[f'{ind}_Trail_Enable'] = 'ON'
        out_cfg[f'{ind}_Trail_Atr'] = float(trail_trig) * PIPS_TO_ATR_FACTOR
    if trail_step is not None:
        out_cfg[f'{ind}_Trail_Step'] = float(trail_step)
        # Trail_Min はデフォルトを維持。必要なら別途設定。

out_cfg.to_csv(OUTPUT_CONFIG, index=False)
print('Wrote config:', OUTPUT_CONFIG)
print('Applied indicators:', TARGET_INDICATORS)
print('Source candidate_id:', rec.get('candidate_id', 'N/A'))
